In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
from scipy import stats
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
metadata = pd.read_csv('../input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

8


In [3]:
df_clusters = pd.DataFrame(index=metadata.index)
for dirpath, dirnames, filenames in os.walk("./"):
    for filename in [f for f in filenames if(f.endswith(".tsv") and f.startswith("clustering"))]:
        print(os.path.join(dirpath, filename))
        df = pd.read_csv(os.path.join(dirpath, filename),sep='\t',index_col=0)
        df_clusters = pd.merge(df_clusters, df, left_index=True, right_index=True)

./Cicero/clusteringSolution.tsv
./cisTopic/clusteringSolution.tsv
./Cusanovich2018/clusteringSolution.tsv
./scABC/clusteringSolution.tsv
./Scasat/clusteringSolution.tsv
./SnapATAC/clusteringSolution.tsv


In [4]:
df_clusters.head()

Cicero  cisTopic  cusanovich2018  scABC  Scasat  SnapATAC
AAACGAAAGCGCAATG-1       3         1               1      5       1         1
AAACGAAAGGGTATCG-1       3         1               1      4       1         1
AAACGAAAGTAACATG-1       6         2               6      7       2         3
AAACGAAAGTTACACC-1       6         2               6      7       3         3
AAACGAACAGAGATGC-1       3         1               1      4       4         1

In [5]:
def residual_average_gini_index(gene_scores,df_clusters,
                                housekeeping_genes,marker_genes,
                                min_cells_per_cluster=10):

    #Subset from the main matrix the housekeeping genes and marker genes
    df_matrix_housekeeping=gene_scores.loc[gene_scores.index.intersection(housekeeping_genes),]
    df_matrix_marker=gene_scores.loc[gene_scores.index.intersection(marker_genes),]
    
    #Define a function to compute the Gini score
    def gini(list_of_values):
        sorted_list = sorted(list_of_values)
        height, area = 0, 0
        for value in sorted_list:
            height += value
            area += height - value / 2.
            fair_area = height * len(list_of_values) / 2.
        return (fair_area - area) / fair_area
    
    #Function to calculate Gini value for all the genes
    def calculate_gini(df_matrix, gene_name,clustering_info):
        return gini(get_avg_per_cluster(df_matrix,gene_name,clustering_info,use_log2=False))

    #Function to calculate Gini value for all the genes
    def calculate_gini_values(df_matrix,clustering_info):
        gini_values=[]
        for gene_name in df_matrix.index:
            gini_values.append(calculate_gini(df_matrix, gene_name,clustering_info))
        return gini_values
    
    #Write a function to compute delta difference of the average accessibility in Marker vs Housekeeping and Kolmogorov Smirnov test
    def score_clustering_solution(df_matrix_marker,df_matrix_housekeeping,clustering_info):
        gini_values_housekeeping=calculate_gini_values(df_matrix_housekeeping,clustering_info)
        gini_values_marker=calculate_gini_values(df_matrix_marker,clustering_info)
        statistic,p_value=stats.ks_2samp(gini_values_marker,gini_values_housekeeping)
        
        return  np.mean(gini_values_marker), np.mean(gini_values_housekeeping),np.mean(gini_values_marker)-np.mean(gini_values_housekeeping), statistic,p_value

    #Function to compute the average accessibility value per cluster
    def get_avg_per_cluster(df_matrix, gene_name, clustering_info,use_log2=False):
        N_clusters=len(clustering_info.index.unique())
        avg_per_cluster=np.zeros(N_clusters)
        for idx,idx_cluster in enumerate(sorted(np.unique(clustering_info.index.unique()))):
            if use_log2:
                values_cluster=df_matrix.loc[gene_name,clustering_info.loc[idx_cluster,:].values.flatten()].apply(lambda x:np.log2(x+1))
            else:
                values_cluster=df_matrix.loc[gene_name,clustering_info.loc[idx_cluster,:].values.flatten()]
           
            avg_per_cluster[idx]=values_cluster.mean()
            if avg_per_cluster[idx]>0:
                  avg_per_cluster[idx]=avg_per_cluster[idx]#/values_cluster.std()

        return avg_per_cluster
    

    #Run the method for all the clustering solutions
    
    df_metrics = pd.DataFrame(columns=['Method','Clustering','Gini_Marker_Genes','Gini_Housekeeping_Genes','Difference','KS_statistics','p-value'])    
    
    for method in df_clusters.columns: 
        print(method)
        df_method_i= df_clusters[method]
        clustering_info = pd.DataFrame(df_method_i)
        clustering_info['Barcode'] = clustering_info.index
        clustering_info=clustering_info.set_index(method)

        #REMOVE CLUSTERS WITH FEW CELLS
        cluster_sizes=pd.value_counts(clustering_info.index)
        clustering_info=clustering_info.loc[cluster_sizes[cluster_sizes>min_cells_per_cluster].index.values,:]


        mean_gini_marker,mean_gini_housekeeping,mean_gini_difference,statistics,p_value=score_clustering_solution(df_matrix_marker,df_matrix_housekeeping,clustering_info)

        df_metrics = df_metrics.append({'Method': method,'Clustering':method,
                           'Gini_Marker_Genes':mean_gini_marker,'Gini_Housekeeping_Genes':mean_gini_housekeeping,
                           'Difference':mean_gini_difference,'KS_statistics':statistics,'p-value':p_value},
                          ignore_index=True)  
        
    return df_metrics

In [6]:
gene_scores = pd.read_csv('../run_methods/GeneScoring/FM_GeneScoring_10xpbmc5k.tsv',
                         sep = '\t',index_col=0)

In [7]:
gene_scores.head()

AAACGAAAGCGCAATG-1  AAACGAAAGGGTATCG-1  AAACGAAAGTAACATG-1  \
A1BG                0.086201            1.775694            3.591624   
A1BG-AS1            0.458201            0.478351            1.365454   
A2LD1               0.000363            0.000000            0.000000   
A2M                 0.000000            0.054960            0.000665   
A2M-AS1             0.000000            0.051265            0.000621   

          AAACGAAAGTTACACC-1  AAACGAACAGAGATGC-1  AAACGAACATGCTATG-1  \
A1BG                0.212587            0.040847            0.004313   
A1BG-AS1            0.930858            0.018763            0.023040   
A2LD1               0.000000            0.000000            0.000000   
A2M                 0.000000            2.205809            0.000000   
A2M-AS1             0.000000            2.057506            0.000000   

          AAACGAAGTGCATCAT-1  AAACGAAGTGGACGAT-1  AAACGAAGTGGCCTCA-1  \
A1BG                0.002157            0.176512            0.000518   
A1BG-AS1            0.011713            0.943315            0.000097   
A2LD1               0.000000            0.000000            0.000000   
A2M                 0.000000            0.000000            0.000000   
A2M-AS1             0.000000            0.000000            0.000000   

          AAACGAATCAGTGTAC-1  ...  TTTGGTTGTCAGAAAT-1  TTTGGTTGTTGTATCG-1  \
A1BG                0.007131  ...            0.041391            3.588490   
A1BG-AS1            0.038674  ...            0.011158            1.345947   
A2LD1               0.000000  ...            0.000000            0.000000   
A2M                 0.000000  ...            0.000000            0.007216   
A2M-AS1             0.000000  ...            0.000000            0.006731   

          TTTGGTTTCAGTGGTT-1  TTTGTGTAGGAAACTT-1  TTTGTGTCAAGCCTTA-1  \
A1BG                1.822400            0.045678            5.371366   
A1BG-AS1            0.922146            0.042287            2.255818   
A2LD1               0.000000            0.000000            0.000000   
A2M                 2.258926            0.000000            0.000819   
A2M-AS1             2.107052            0.000000            0.000764   

          TTTGTGTCACTCAAGT-1  TTTGTGTCACTGGGCT-1  TTTGTGTGTACGCAAG-1  \
A1BG                0.002675            2.454377            1.984945   
A1BG-AS1            0.012003            3.702715            1.393866   
A2LD1               0.000000            0.000000            0.000000   
A2M                 0.000000            0.001228            0.000665   
A2M-AS1             0.000000            0.001146            0.000621   

          TTTGTGTGTCTGCGCA-1  TTTGTGTTCAACTTGG-1  
A1BG                1.732547            1.730391  
A1BG-AS1            0.444438            0.432918  
A2LD1               0.000000            0.000000  
A2M                 0.000000            0.000000  
A2M-AS1             0.000000            0.000000  

[5 rows x 5335 columns]

In [8]:
#https://www.tau.ac.il/~elieis/Housekeeping_genes.html List of Housekeeping genes
housekeeping_genes=['ACTB','ALDOA','GAPDH','PGK1','LDHA','RPS27A','RPL19','RPL11','NONO','ARHGDIA','RPL32','RPS18','HSPCB',
    'C1orf43','CHMP2A','EMC7','GPI','PSMB2,''PSMB4','RAB7A','REEP5','SNRPD3','VCP','VPS29']

#List of Marker Genes
marker_genes=['CD209', 'ENG', 'FOXP3', 'CD34', 'BATF3', 'S100A12', 'THBD','CD3D', 'THY1', 'CD8A', 'CD8B', 'CD14', 'PROM1', 'IL2RA', 'FCGR3A',
'IL3RA', 'FCGR1A', 'CD19', 'IL7R', 'CD79A', 'MS4A1', 'NCAM1','CD3E', 'CD3G', 'KIT', 'CD1C', 'CD68', 'CD4']

In [9]:
df_metrics = residual_average_gini_index(gene_scores,df_clusters,
                                housekeeping_genes,marker_genes,
                                min_cells_per_cluster=10)

Cicero
cisTopic
cusanovich2018
scABC
Scasat
SnapATAC


In [10]:
df_metrics

Method      Clustering  Gini_Marker_Genes  Gini_Housekeeping_Genes  \
0          Cicero          Cicero           0.438122                 0.360182   
1        cisTopic        cisTopic           0.468534                 0.362493   
2  cusanovich2018  cusanovich2018           0.414134                 0.277127   
3           scABC           scABC           0.429966                 0.319984   
4          Scasat          Scasat           0.487809                 0.399358   
5        SnapATAC        SnapATAC           0.379297                 0.237219   

   Difference  KS_statistics   p-value  
0    0.077940       0.464286  0.007129  
1    0.106041       0.500000  0.002897  
2    0.137006       0.500000  0.002897  
3    0.109982       0.535714  0.001101  
4    0.088451       0.488095  0.003940  
5    0.142078       0.571429  0.000392

In [11]:
df_metrics.to_csv('./clustering_RAGI_scores.csv')